In [1]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import Embedding, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [4]:
total_words = 10000
batch_size = 128
max_review_len = 80
embedding_len = 100

In [5]:
(x_train, y_train), (x_test,y_test) = imdb.load_data(num_words = total_words)

In [6]:
print(x_train.shape)
print(x_test.shape)

(25000,)
(25000,)


In [7]:
pad_x_train = sequence.pad_sequences(x_train, maxlen= max_review_len)
pad_x_test = sequence.pad_sequences(x_test, maxlen= max_review_len)

In [8]:
print(pad_x_train.shape)
print(pad_x_test.shape)

(25000, 80)
(25000, 80)


In [16]:
train_data = tf.data.Dataset.from_tensor_slices((pad_x_train,y_train))
train_data = train_data.shuffle(10000).batch(batch_size, drop_remainder=True)
test_data = tf.data.Dataset.from_tensor_slices((pad_x_test,y_test))
test_data = test_data.batch(batch_size, drop_remainder=True)
print('x_train_shape:', pad_x_train.shape, tf.reduce_max(y_train), tf.reduce_min(y_train))
print('x_test_shape:', pad_x_test.shape)

x_train_shape: (25000, 80) tf.Tensor(1, shape=(), dtype=int64) tf.Tensor(0, shape=(), dtype=int64)
x_test_shape: (25000, 80)


In [ ]:
class LSTM_Build(tf.keras.Model):
    def __init__(self, units):
        super(LSTM_Build, self).__init__()
        
        self.state0 = [tf.zeros([batch_size, units]), tf.zeros([batch_size,units])]
        self.state1 = [tf.zeros([batch_size, units]), tf.zeros([batch_size,units])]
        
        self.embedding = tf.keras.layers.Embedding(total_words, embedding_len, input_length=max_review_len)
        self.RNNCell0 = tf.keras.layers.LSTMCell(units, dropout=0.5)
        self.RNNCell1 = tf.keras.layers.LSTMCell(units, dropout=0.5)
        self.outlayer = tf.keras.layers.Dense(1)
        
    def call(self, inputs, training=None):
        x = inputs
        x = self.embedding(x)
        state0 = self.state0
        state1 = self.state1
        for word in tf.unstack(x, axis=1):
            out0, state0 = self.RNNCell0(word, state0, training)
            out1, state1 = self.RNNCell1(out0, state1, training)
        x = self.outlayer(out1)
        prob = tf.sigmoid(x)
        return prob

In [ ]:
import time
units = 64
epochs = 4
t0 = time.time()

model = LSTM_Build(units)

model.compile(optimizer=Adam(0.001),
             loss=tf.losses.BinaryCrossentropy(),
             metrics=['accuracy'],
             experimental_run_tf_function=False)

In [ ]:
model.fit(train_data, epochs=epochs, validation_data=test_data, validation_freq=2)

In [38]:
print("훈련 데이터셋 평가...")
(loss, accuracy) = model.evaluate(train_data, verbose=0)
print("loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))
print("테스트 데이터셋 평가...")
(loss, accuracy) = model.evaluate(test_data, verbose=0)
print("loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))
t1 = time.time()
print('시간:', t1-t0)

훈련 데이터셋 평가...
loss=0.1690, accuracy: 93.7180%
테스트 데이터셋 평가...
loss=0.4099, accuracy: 82.5521%
시간: 116.50875282287598


In [52]:
class LSTM_Build(tf.keras.Model):
    def __init__(self, units):
        super(LSTM_Build, self).__init__()
        
        self.state0 = [tf.zeros([batch_size, units]), tf.zeros([batch_size,units])]
        self.state1 = [tf.zeros([batch_size, units]), tf.zeros([batch_size,units])]
        
        self.embedding = tf.keras.layers.Embedding(total_words, embedding_len, input_length=max_review_len)
        self.rnn = tf.keras.Sequential([
            tf.keras.layers.LSTM(units, dropout=0.5, return_sequences=True, unroll=True),
            tf.keras.layers.LSTM(units, dropout=0.5, unroll=True)])
        self.outlayer = tf.keras.layers.Dense(1)
        
    def call(self, inputs, training=None):
        x = inputs
        x = self.embedding(x)
        x = self.rnn(x)
        x = self.outlayer(x)
        prob = tf.sigmoid(x)
        return prob

In [53]:
import time
units = 64
epochs = 4
t0 = time.time()

model = LSTM_Build(units)

model.compile(optimizer=Adam(0.001),
             loss=tf.losses.BinaryCrossentropy(),
             metrics=['accuracy'],
             experimental_run_tf_function=False)

In [54]:
model.fit(train_data, epochs=epochs, validation_data=test_data, validation_freq=2)

Epoch 1/4
195/195 [==============================] - 36s 148ms/step - loss: 0.4806 - accuracy: 0.7550
Epoch 2/4
195/195 [==============================] - 40s 205ms/step - loss: 0.3176 - accuracy: 0.8687 - val_loss: 0.3670 - val_accuracy: 0.8356
Epoch 3/4
195/195 [==============================] - 29s 150ms/step - loss: 0.2599 - accuracy: 0.8976
Epoch 4/4
195/195 [==============================] - 40s 203ms/step - loss: 0.2186 - accuracy: 0.9141 - val_loss: 0.4603 - val_accuracy: 0.8333


In [55]:
print("훈련 데이터셋 평가...")
(loss, accuracy) = model.evaluate(train_data, verbose=0)
print("loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))
print("테스트 데이터셋 평가...")
(loss, accuracy) = model.evaluate(test_data, verbose=0)
print("loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))
t1 = time.time()
print('시간:', t1-t0)

훈련 데이터셋 평가...
loss=0.1417, accuracy: 95.0761%
테스트 데이터셋 평가...
loss=0.4603, accuracy: 83.3333%
시간: 165.08115935325623


In [56]:
model.summary()

Model: "lstm__build_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_13 (Embedding)    multiple                  1000000   
                                                                 
 sequential_1 (Sequential)   (128, 64)                 75264     
                                                                 
 dense_8 (Dense)             multiple                  65        
                                                                 
Total params: 1,075,329
Trainable params: 1,075,329
Non-trainable params: 0
_________________________________________________________________


In [57]:
class GRU_Build(tf.keras.Model):
    def __init__(self, units):
        super(GRU_Build, self).__init__()
        
        self.state0 = [tf.zeros([batch_size, units]), tf.zeros([batch_size,units])]
        self.state1 = [tf.zeros([batch_size, units]), tf.zeros([batch_size,units])]
        
        self.embedding = tf.keras.layers.Embedding(total_words, embedding_len, input_length=max_review_len)
        self.RNNCell0 = tf.keras.layers.GRUCell(units, dropout=0.5)
        self.RNNCell1 = tf.keras.layers.GRUCell(units, dropout=0.5)
        self.outlayer = tf.keras.layers.Dense(1)
        
    def call(self, inputs, training=None):
        x = inputs
        x = self.embedding(x)
        state0 = self.state0
        state1 = self.state1
        for word in tf.unstack(x, axis=1):
            out0, state0 = self.RNNCell0(word, state0, training)
            out1, state1 = self.RNNCell1(out0, state1, training)
        x = self.outlayer(out1)
        prob = tf.sigmoid(x)
        return prob

In [58]:
import time
units = 64
epochs = 4
t0 = time.time()

model = GRU_Build(units)

model.compile(optimizer=Adam(0.001),
             loss=tf.losses.BinaryCrossentropy(),
             metrics=['accuracy'],
             experimental_run_tf_function=False)

In [59]:
model.fit(train_data, epochs=epochs, validation_data=test_data, validation_freq=2)

Epoch 1/4
195/195 [==============================] - 57s 248ms/step - loss: 0.5108 - accuracy: 0.7271
Epoch 2/4
195/195 [==============================] - 64s 329ms/step - loss: 0.3232 - accuracy: 0.8635 - val_loss: 0.3590 - val_accuracy: 0.8409
Epoch 3/4
195/195 [==============================] - 52s 267ms/step - loss: 0.2631 - accuracy: 0.8956
Epoch 4/4
195/195 [==============================] - 67s 345ms/step - loss: 0.2266 - accuracy: 0.9120 - val_loss: 0.4437 - val_accuracy: 0.8298


In [62]:
print("GRU 훈련 데이터셋 평가...")
(loss, accuracy) = model.evaluate(train_data, verbose=0)
print("loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))
print("GRU 테스트 데이터셋 평가...")
(loss, accuracy) = model.evaluate(test_data, verbose=0)
print("loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))
t1 = time.time()
print('시간:', t1-t0)

GRU 훈련 데이터셋 평가...
loss=0.1464, accuracy: 94.6995%
GRU 테스트 데이터셋 평가...
loss=0.4437, accuracy: 82.9848%
시간: 339.1012153625488


In [63]:
class GRU_Build(tf.keras.Model):
    def __init__(self, units):
        super(GRU_Build, self).__init__()
        
        self.state0 = [tf.zeros([batch_size, units]), tf.zeros([batch_size,units])]
        self.state1 = [tf.zeros([batch_size, units]), tf.zeros([batch_size,units])]
        
        self.embedding = tf.keras.layers.Embedding(total_words, embedding_len, input_length=max_review_len)
        self.rnn = tf.keras.Sequential([
            tf.keras.layers.GRU(units, dropout=0.5, return_sequences=True, unroll=True),
            tf.keras.layers.GRU(units, dropout=0.5, unroll=True)])
        self.outlayer = tf.keras.layers.Dense(1)
        
    def call(self, inputs, training=None):
        x = inputs
        x = self.embedding(x)
        x = self.rnn(x)
        x = self.outlayer(x)
        prob = tf.sigmoid(x)
        return prob

In [64]:
import time
units = 64
epochs = 4
t0 = time.time()

model = GRU_Build(units)

model.compile(optimizer=Adam(0.001),
             loss=tf.losses.BinaryCrossentropy(),
             metrics=['accuracy'],
             experimental_run_tf_function=False)

In [65]:
model.fit(train_data, epochs=epochs, validation_data=test_data, validation_freq=2)

Epoch 1/4
195/195 [==============================] - 73s 324ms/step - loss: 0.5131 - accuracy: 0.7252
Epoch 2/4
195/195 [==============================] - 79s 406ms/step - loss: 0.3257 - accuracy: 0.8627 - val_loss: 0.3747 - val_accuracy: 0.8373
Epoch 3/4
195/195 [==============================] - 62s 319ms/step - loss: 0.2731 - accuracy: 0.8906
Epoch 4/4
195/195 [==============================] - 83s 425ms/step - loss: 0.2412 - accuracy: 0.9070 - val_loss: 0.3723 - val_accuracy: 0.8362


In [66]:
print("GRU 훈련 데이터셋 평가...")
(loss, accuracy) = model.evaluate(train_data, verbose=0)
print("loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))
print("GRU 테스트 데이터셋 평가...")
(loss, accuracy) = model.evaluate(test_data, verbose=0)
print("loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))
t1 = time.time()
print('시간:', t1-t0)

GRU 훈련 데이터셋 평가...
loss=0.1901, accuracy: 94.2428%
GRU 테스트 데이터셋 평가...
loss=0.3723, accuracy: 83.6218%
시간: 334.7041964530945
